# هسته معنایی با یکپارچه‌سازی سرور OpenBnB MCP

این دفترچه نشان می‌دهد که چگونه می‌توان از هسته معنایی با سرور واقعی OpenBnB MCP برای جستجوی اقامتگاه‌های واقعی Airbnb با استفاده از MCPStdioPlugin استفاده کرد. برای دسترسی به LLM، از Azure AI Foundry استفاده می‌شود. برای تنظیم متغیرهای محیطی خود، می‌توانید درس [تنظیمات](/00-course-setup/README.md) را دنبال کنید.


## وارد کردن بسته‌های مورد نیاز


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio

from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## ایجاد اتصال پلاگین MCP

ما با استفاده از MCPStdioPlugin به [سرور MCP اوپن‌بی‌ان‌بی](https://github.com/openbnb-org/mcp-server-airbnb) متصل خواهیم شد. این سرور قابلیت جستجوی Airbnb را از طریق بسته @openbnb/mcp-server-airbnb فراهم می‌کند.


## ایجاد کلاینت

در این نمونه، ما از Azure AI Foundry برای دسترسی به LLM استفاده خواهیم کرد. مطمئن شوید که متغیرهای محیطی شما به درستی تنظیم شده‌اند.


## پیکربندی محیط

تنظیمات Azure OpenAI را پیکربندی کنید. اطمینان حاصل کنید که متغیرهای محیطی زیر تنظیم شده‌اند:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## درک یکپارچه‌سازی OpenBnB MCP

این دفترچه به **سرور واقعی OpenBnB MCP** متصل می‌شود که قابلیت جستجوی واقعی Airbnb را فراهم می‌کند.

### نحوه عملکرد:

1. **MCPStdioPlugin**: از ارتباط ورودی/خروجی استاندارد با سرور MCP استفاده می‌کند
2. **پکیج واقعی NPM**: پکیج `@openbnb/mcp-server-airbnb` را از طریق npx دانلود و اجرا می‌کند
3. **داده‌های زنده**: اطلاعات واقعی املاک Airbnb را از API‌های آن‌ها بازمی‌گرداند
4. **کشف عملکردها**: عامل به‌طور خودکار عملکردهای موجود در سرور MCP را کشف می‌کند

### عملکردهای موجود:

سرور OpenBnB MCP معمولاً این عملکردها را ارائه می‌دهد:
- **search_listings** - جستجوی املاک Airbnb بر اساس مکان و معیارها
- **get_listing_details** - دریافت اطلاعات دقیق درباره املاک خاص
- **check_availability** - بررسی موجود بودن برای تاریخ‌های مشخص
- **get_reviews** - دریافت نظرات درباره املاک
- **get_host_info** - دریافت اطلاعات درباره میزبان‌های املاک

### پیش‌نیازها:

- **Node.js** نصب شده بر روی سیستم شما
- **اتصال به اینترنت** برای دانلود پکیج سرور MCP
- **NPX** در دسترس (همراه با Node.js ارائه می‌شود)

### آزمایش اتصال:

می‌توانید سرور MCP را به‌صورت دستی با اجرای دستور زیر آزمایش کنید:
```bash
npx -y @openbnb/mcp-server-airbnb
```

این دستور سرور OpenBnB MCP را دانلود و اجرا می‌کند، که سپس Semantic Kernel به آن متصل می‌شود تا داده‌های واقعی Airbnb را دریافت کند.


## اجرای عامل با سرور OpenBnB MCP

اکنون عامل هوش مصنوعی را اجرا می‌کنیم که به سرور OpenBnB MCP متصل می‌شود تا اقامتگاه‌های واقعی Airbnb در استکهلم برای ۲ بزرگسال و ۱ کودک جستجو کند. می‌توانید لیست `user_inputs` را تغییر دهید تا معیارهای جستجو را اصلاح کنید.


In [ ]:
# Main execution cell - Enhanced with proper HTML rendering and MCP tool logging
# User requests for Airbnb search
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        # Create MCP plugin connection to real OpenBnB server
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb", "--ignore-robots-txt"],
        ) as airbnb_plugin:

            print("🔧 MCP Plugin created and connected")

            # Load tools for function discovery
            await airbnb_plugin.load_tools()
            await asyncio.sleep(3)  # Give more time for initialization
            print("✅ Tools loaded from MCP server")

            # Debug: Check what tools were loaded
            if hasattr(airbnb_plugin, '_tools'):
                print(f"📋 Internal tools: {airbnb_plugin._tools}")

            # Verify available functions
            funcs = [attr for attr in dir(airbnb_plugin)
                     if callable(getattr(airbnb_plugin, attr))
                     and attr in ['airbnb_search', 'airbnb_listing_details']]
            print(f"📋 Available functions: {funcs}")

            # Create agent with Azure OpenAI service
            agent = ChatCompletionAgent(
                service=AzureChatCompletion(),  # Use default constructor
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the airbnb_search function to find properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("🤖 Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 Processing request: {user_input}")
                
                # Track MCP tool usage
                mcp_tools_used = []
                function_calls_log = []
                
                # Try streaming to capture function calls
                try:
                    agent_name = None
                    full_response = []
                    current_function_name = None
                    argument_buffer = ""
                    
                    async for response in agent.invoke_stream(
                        messages=user_input,
                        thread=thread,
                    ):
                        thread = response.thread
                        agent_name = response.name
                        
                        for item in response.items:
                            # Log function calls
                            if isinstance(item, FunctionCallContent):
                                if item.function_name:
                                    current_function_name = item.function_name
                                    mcp_tools_used.append(item.function_name)
                                    print(f"\n🔧 MCP Tool Selected: {item.function_name}")
                                    
                                if isinstance(item.arguments, str):
                                    argument_buffer += item.arguments
                            
                            # Log function results
                            elif isinstance(item, FunctionResultContent):
                                if current_function_name:
                                    try:
                                        args = json.loads(argument_buffer.strip()) if argument_buffer else {}
                                    except:
                                        args = {"raw": argument_buffer}
                                    
                                    function_calls_log.append({
                                        "function": current_function_name,
                                        "arguments": args,
                                        "timestamp": asyncio.get_event_loop().time()
                                    })
                                    
                                    print(f"   📍 Arguments: {json.dumps(args, indent=2)}")
                                    print(f"   ✅ MCP Tool Executed Successfully")
                                    
                                    current_function_name = None
                                    argument_buffer = ""
                            
                            # Collect response text
                            elif isinstance(item, StreamingTextContent) and item.text:
                                full_response.append(item.text)
                    
                    # Join the full response
                    response_text = ''.join(full_response)
                    
                except Exception as e:
                    print(f"⚠️ Streaming failed, using get_response: {str(e)[:100]}")
                    # Fallback to non-streaming
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    response_text = str(response)
                    agent_name = response.name
                
                
                # Process the response to ensure HTML tables render correctly
                # Remove any markdown code blocks around HTML
                response_text = response_text.replace('```html', '').replace('```', '')
                
                # Ensure proper HTML structure for tables
                if '<table' in response_text.lower():
                    # Add CSS styling for better table rendering
                    table_css = """
                    <style>
                        .airbnb-results table {
                            border-collapse: collapse;
                            width: 100%;
                            margin: 10px 0;
                        }
                        .airbnb-results th, .airbnb-results td {
                            border: 1px solid #ddd;
                            padding: 8px;
                            text-align: left;
                        }
                        .airbnb-results th {
                            background-color: #f2f2f2;
                            font-weight: bold;
                        }
                        .airbnb-results tr:nth-child(even) {
                            background-color: #f9f9f9;
                        }
                        .airbnb-results a {
                            color: #1976d2;
                            text-decoration: none;
                        }
                        .airbnb-results a:hover {
                            text-decoration: underline;
                        }
                    </style>
                    """
                    response_text = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                
                # Build the complete HTML output
                html_output = f"""
                <div style='margin:10px; padding:10px; border-left:3px solid #2E8B57; background:#F0F8FF;'>
                    <strong>User:</strong> {user_input}
                </div>
                """
                
                # Add function call details if available
                if function_calls_log:
                    details_html = "<details style='margin:10px; padding:10px; background:#f5f5f5;'>"
                    details_html += "<summary><strong>📊 Function Call Details</strong></summary>"
                    details_html += "<pre style='background:#fff; padding:10px; overflow-x:auto;'>"
                    for call in function_calls_log:
                        details_html += f"Function: {call['function']}\n"
                        details_html += f"Arguments: {json.dumps(call['arguments'], indent=2)}\n"
                        details_html += "---\n"
                    details_html += "</pre></details>"
                    html_output += details_html
                
                # Add the agent's response with proper HTML rendering
                html_output += f"""
                <div style='margin:10px; padding:15px; border-left:3px solid #1E90FF; background:#FFFFFF;'>
                    <strong>{agent_name}:</strong><br>
                    {response_text}
                </div>
                """
                
                # Display the HTML with proper rendering
                display(HTML(html_output))
                
                
    except Exception as e:
        print(f"❌ Error: {str(e)}")
        import traceback
        traceback.print_exc()

print("🚀 Starting with Azure OpenAI...")
await main()
print("✅ Done!")

خلاصه  
تبریک می‌گوییم! شما با موفقیت یک عامل هوش مصنوعی ساخته‌اید که با استفاده از پروتکل Model Context Protocol (MCP) به جستجوی اقامتگاه‌های واقعی در دنیای واقعی متصل می‌شود:

فناوری‌های استفاده‌شده:  
- Semantic Kernel - برای ساخت عوامل هوشمند با Azure OpenAI  
- Azure AI Foundry - برای قابلیت‌های مدل‌های زبانی بزرگ (LLM) و تکمیل چت  
- MCP (Model Context Protocol) - برای یکپارچه‌سازی استاندارد ابزارها  
- OpenBnB MCP Server - برای قابلیت جستجوی واقعی Airbnb  
- Node.js/NPX - برای اجرای سرور MCP خارجی  

آنچه آموخته‌اید:  
- یکپارچه‌سازی MCP: اتصال عوامل Semantic Kernel به سرورهای خارجی MCP  
- دسترسی به داده‌های بلادرنگ: جستجوی املاک واقعی Airbnb از طریق APIهای زنده  
- ارتباط پروتکلی: استفاده از ارتباط stdio بین عامل و سرور MCP  
- کشف عملکرد: کشف خودکار توابع موجود از سرورهای MCP  
- پاسخ‌های جریانی: ثبت و ضبط تماس‌های توابع در زمان واقعی  
- رندر HTML: قالب‌بندی پاسخ‌های عامل با جداول استایل‌دهی‌شده و نمایش‌های تعاملی  

گام‌های بعدی:  
- یکپارچه‌سازی سرورهای MCP اضافی (آب‌وهوا، پروازها، رستوران‌ها)  
- ساخت یک سیستم چندعاملی با ترکیب پروتکل‌های MCP و A2A  
- ایجاد سرورهای MCP سفارشی برای منابع داده خود  
- پیاده‌سازی حافظه مکالمه پایدار در طول جلسات  
- استقرار عامل در Azure Functions با هماهنگی سرور MCP  
- افزودن قابلیت‌های احراز هویت کاربر و رزرو  

مزایای کلیدی معماری MCP:  
- استانداردسازی: پروتکل جهانی برای اتصال عوامل هوش مصنوعی به ابزارهای خارجی  
- داده‌های بلادرنگ: دسترسی به اطلاعات زنده و به‌روز از خدمات مختلف  
- توسعه‌پذیری: یکپارچه‌سازی آسان منابع داده و ابزارهای جدید  
- قابلیت همکاری: کارکرد در چارچوب‌های مختلف هوش مصنوعی و پلتفرم‌های عامل  
- جداسازی نگرانی‌ها: تمایز واضح بین منطق هوش مصنوعی و دسترسی به داده‌های خارجی  



---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
